# Práce s datem a časem

Dnes budeme pracovat se soubory [invoices.csv](invoices.csv), [invoices_2.csv](invoices_2.csv) a [signal_monitoring.csv](signal_monitoring.csv).

Zkusme si načíst následující tabulku.

In [1]:
import pandas

invoices = pandas.read_csv("invoices.csv")
invoices.head()

,invoice_date,customer,amount
0,24. 07. 2021,Robbins-Bryant,42424.49
1,03. 06. 2021,"Little, Espinoza and Manning",6269.53
2,24. 06. 2021,Hartman-Garcia,19182.55
3,09. 08. 2021,"Mason, Armstrong and Whitaker",66295.23
4,29. 06. 2021,Dickerson-Blankenship,88592.58


Podívejme se nyní na to, jaké datové typy mají jednotlivé sloupce.

In [2]:
invoices.dtypes

invoice_date     object
customer         object
amount          float64
dtype: object

První dva sloupce mají obecný typ `object`, `pandas` tedy nerozpoznal, že v prvním sloupci je datum. To je škoda, protože nám to neumožní s datem pracovat. Můžeme ale hodnotu na typ `datetime` převést. K převodu použijeme funkci `to_datetime`, které předáme sloupec `invoice_date` a výstup uložíme do sloupce `invoice_date_converted`.

In [3]:
invoices["invoice_date_converted"] = pandas.to_datetime(invoices["invoice_date"])
invoices.head()

,invoice_date,customer,amount,invoice_date_converted
0,24. 07. 2021,Robbins-Bryant,42424.49,2021-07-24
1,03. 06. 2021,"Little, Espinoza and Manning",6269.53,2021-03-06
2,24. 06. 2021,Hartman-Garcia,19182.55,2021-06-24
3,09. 08. 2021,"Mason, Armstrong and Whitaker",66295.23,2021-09-08
4,29. 06. 2021,Dickerson-Blankenship,88592.58,2021-06-29


Překvapivé je, že si pandas s českým formátem poradil. Resp. se to tak zdá. Pokud si ale sloupce pečlivě porovnáme, zjistíme, že v případě řádku 1 byl 3. červen transformován na 6. březen.

Pandas totiž předpokládá **americký styl zápisu**, který má (z nepochopitelných důvodů) na prvním místě měsíc a na druhém místě den. Není však dogmatický a u 2 řádku pochopil, že 27 číslo měsíce není a datum převede správně. Již mu ale nedojde, že by mohl stejným stylem převést ostatní data (a ostatně ani není zaručeno, že všechna data musí mít stejný formát).

První možností, jak si s tím poradit, je nastavení parametru `dayfirst`, kterým říkáme, že číslo jde je před číslem měsíce. Je zřejmé, že výchozí hodnota parametru je `False`.

In [4]:
invoices["invoice_date_converted"] = pandas.to_datetime(invoices["invoice_date"], dayfirst=True)
invoices.head()

,invoice_date,customer,amount,invoice_date_converted
0,24. 07. 2021,Robbins-Bryant,42424.49,2021-07-24
1,03. 06. 2021,"Little, Espinoza and Manning",6269.53,2021-06-03
2,24. 06. 2021,Hartman-Garcia,19182.55,2021-06-24
3,09. 08. 2021,"Mason, Armstrong and Whitaker",66295.23,2021-08-09
4,29. 06. 2021,Dickerson-Blankenship,88592.58,2021-06-29


## Přičítání a porovnávání

Každá faktura má nějaký datum splatnosti. Nyní chceme přidat sloupec s datem splatnosti s tím, že datum splatnosti je vždy 60 dní po vystavení faktury. K tomu využijeme třídu `Timedelta` modulu `pandas`.

Třída umožňuje poměrně jednoduché vytvoření hodnoty zápisem `"60 days"` coby řetězce. Alternativně můžeme použít [ISO normu](https://en.wikipedia.org/wiki/ISO_8601#Durations) pro zápis délky trvání a zapsat "P60D".

In [5]:
invoices["due_date"] = invoices["invoice_date_converted"] + pandas.Timedelta("P60D")
invoices.head()

,invoice_date,customer,amount,invoice_date_converted,due_date
0,24. 07. 2021,Robbins-Bryant,42424.49,2021-07-24,2021-09-22
1,03. 06. 2021,"Little, Espinoza and Manning",6269.53,2021-06-03,2021-08-02
2,24. 06. 2021,Hartman-Garcia,19182.55,2021-06-24,2021-08-23
3,09. 08. 2021,"Mason, Armstrong and Whitaker",66295.23,2021-08-09,2021-10-08
4,29. 06. 2021,Dickerson-Blankenship,88592.58,2021-06-29,2021-08-28


Pojďme nyní roztřídit faktury na ty, které jsou před splatností a ty, které jsou po splatnosti. Třídění budeme provádět k 1. září 2021. Vytvoříme tedy objekt, který reprezentuje toto datum (využijeme standardní modul `datetime`) a funkci `where()` z modulu `numpy`, která slouží k zápisu podmínky.

Přidáme sloupec `status`, který bude obsahovat texty `overdue` a `before due date`.

In [6]:
import numpy
import datetime
today_date = datetime.datetime(2021, 9, 1)
invoices["status"] = numpy.where(invoices["due_date"] < today_date, "overdue", "before due date")
invoices.head()

,invoice_date,customer,amount,invoice_date_converted,due_date,status
0,24. 07. 2021,Robbins-Bryant,42424.49,2021-07-24,2021-09-22,before due date
1,03. 06. 2021,"Little, Espinoza and Manning",6269.53,2021-06-03,2021-08-02,overdue
2,24. 06. 2021,Hartman-Garcia,19182.55,2021-06-24,2021-08-23,overdue
3,09. 08. 2021,"Mason, Armstrong and Whitaker",66295.23,2021-08-09,2021-10-08,before due date
4,29. 06. 2021,Dickerson-Blankenship,88592.58,2021-06-29,2021-08-28,overdue


Nakonec můžeme spočítat, jaké finanční prostředky jsou ve fakturách před splatností a jaké prostředky v nezaplacených fakturách po splatnosti.

In [7]:
invoices.groupby("status")["amount"].sum()

status
before due date    2.383203e+09
overdue            2.604655e+09
Name: amount, dtype: float64

## Výpočty s daty

Máme-li k dispozici dva různé sloupce s daty, můžeme s nimi provádět výpočty, nejčastěji počítat rozdíl, tj. délku časového intervalu mezi nimi.

Uvažujme, že nyní máme k dispozici delší časovou řadu faktur. Nejprve spočítáme, po kolika dnech v průměru zaplatí fakturu jednotliví zákazníci. Tuto hodnotu pak použijeme k výpočtu očekávané splatnosti nových faktur.

Nejprve načteme tabulku s fakturami.

In [8]:
invoices_2 = pandas.read_csv("invoices_2.csv")
invoices_2["invoice_date"] = pandas.to_datetime(invoices_2["invoice_date"], dayfirst=True)
invoices_2.head()

,invoice_date,customer,amount,payment_date
0,2016-05-01,Campos PLC,29366,25. 06. 2016
1,2016-05-10,"Garcia, Hamilton and Carr",61145,16. 07. 2016
2,2016-05-20,"Garcia, Hamilton and Carr",92659,26. 07. 2016
3,2016-05-25,"Nguyen, Harrell and Lee",40076,05. 08. 2016
4,2016-06-04,"Garcia, Hamilton and Carr",54854,06. 08. 2016


V první části se budeme zabývat splacenými fakturami. Splacené faktury mají prázdnou hodnotu ve sloupci `payment_date`, vyfiltrujeme je pomocí metody `dropna`. Čas, který zákazník potřeboval na splacení faktury, získáme pomocí rozdílu sloupců `payment_date` a `invoice_date`.

In [9]:
invoices_2_paid = invoices_2.dropna().reset_index(drop=True)
invoices_2_paid["payment_date"] = pandas.to_datetime(invoices_2_paid["payment_date"], dayfirst=True)
invoices_2_paid["paid_in"] = invoices_2_paid["payment_date"] - invoices_2_paid["invoice_date"]
invoices_2_paid.head()

,invoice_date,customer,amount,payment_date,paid_in
0,2016-05-01,Campos PLC,29366,2016-06-25,55 days
1,2016-05-10,"Garcia, Hamilton and Carr",61145,2016-07-16,67 days
2,2016-05-20,"Garcia, Hamilton and Carr",92659,2016-07-26,67 days
3,2016-05-25,"Nguyen, Harrell and Lee",40076,2016-08-05,72 days
4,2016-06-04,"Garcia, Hamilton and Carr",54854,2016-08-06,63 days


Hodnota rozdílu je uložené jako datový typ `timedelta`.

In [10]:
invoices_2_paid.dtypes

invoice_date     datetime64[ns]
customer                 object
amount                    int64
payment_date     datetime64[ns]
paid_in         timedelta64[ns]
dtype: object

Nyní provedeme agregaci dle zákazníka a zjistíme tak průměrnou dobu splatnosti faktury.

In [11]:
average_payment_data = pandas.DataFrame(invoices_2_paid.groupby(["customer"])["paid_in"].mean())
average_payment_data.head()

,paid_in
customer,
Archer-Patel,46 days 20:34:17.142857143
Campos PLC,60 days 00:00:00
Chang-Fisher,63 days 05:20:00
Clayton-Cook,69 days 22:09:13.846153846
Faulkner-Howard,74 days 06:32:43.636363636


### Dokončení příkladu

*Níže je dokončení příkladu, které je ale spíše opakováním (je tam agregace a propojení tabulek). Na lekci ho procházet nebudeme, ale určitě se k němu můžete vrátit v rámci čtení na doma a zopakovat si na něm věcí z kurzu Python pro data 1 i dnešní lekce.*

Nyní budeme pracovat s nesplacenými fakturami, které získáme např. dotazem s využitím metody `isna()`. Poté propojíme tabulku s nesplacenými fakturami s tabulkou průměrných splatností a přičteme průměrnou dobu splatnosti k datu vystavení faktury. Abychom získali splatnost jako den, použijeme vlastnost `dt.date`.

In [12]:
invoices_2_not_paid = invoices_2[invoices_2["payment_date"].isna()]
invoices_2_not_paid = pandas.merge(invoices_2_not_paid, average_payment_data, on=["customer"])
invoices_2_not_paid["expected_payment_date"] = invoices_2_not_paid["invoice_date"] + pandas.to_timedelta(invoices_2_not_paid["paid_in"], unit="D")
invoices_2_not_paid["expected_payment_date"] = invoices_2_not_paid["expected_payment_date"].dt.date
invoices_2_not_paid.head()

,invoice_date,customer,amount,payment_date,paid_in,expected_payment_date
0,2021-07-02,"Nguyen, Harrell and Lee",93301,NaN,67 days 05:08:34.285714286,2021-09-07
1,2021-07-09,Levy Group,43621,NaN,65 days 00:00:00,2021-09-12
2,2021-08-05,Levy Group,39651,NaN,65 days 00:00:00,2021-10-09
3,2021-07-16,Wagner LLC,65064,NaN,58 days 05:08:34.285714286,2021-09-12
4,2021-07-22,"Jones, Jefferson and Rivera",98317,NaN,71 days 15:00:00,2021-10-01


# Čtení na doma


## Přesný formát data a času

Konzervativnější přístup je ale přesná definice formáta data, který využívá stejné symboly, jako funkce `strftime` modulu `datetime`. Náš český formát bychom zapsatli jako `"%d. %m. %Y"`:

- `%d` zastupuje číslo dne,
- `%m` číslo měsíce a
- `%Y` rok.

Kompletní význam všech zástupných symbolů je stejný jako v jazyce Python a najdeme jen v [dokumentaci modulu datetime](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior).

In [22]:
invoices["invoice_date_converted"] = pandas.to_datetime(invoices["invoice_date"], format="%d. %m. %Y")
invoices.head()

,invoice_date,customer,amount,invoice_date_converted,due_date,status
0,24. 07. 2021,Robbins-Bryant,42424.49,2021-07-24,2021-09-22,before due date
1,03. 06. 2021,"Little, Espinoza and Manning",6269.53,2021-06-03,2021-08-02,overdue
2,24. 06. 2021,Hartman-Garcia,19182.55,2021-06-24,2021-08-23,overdue
3,09. 08. 2021,"Mason, Armstrong and Whitaker",66295.23,2021-08-09,2021-10-08,before due date
4,29. 06. 2021,Dickerson-Blankenship,88592.58,2021-06-29,2021-08-28,overdue


Funkce `to_datetime` tiše dělá ještě jednu chytrou věc. Převod data z řetězce totiž není úplně jednoduchá záležitost a i když ji moderní počítač zvládne velmi rychle, stále platí pravidlo, že tisíckrát nic umořilo osla. Pokud by tedy `pandas` převáděl každý řetězec, operace by trvala dlouho.

Funkce `to_datetime` na to ale jde chytřeji a používá vyrovnávají paměť nebo-li `cache`. Každý převedený řetězec si uloží a předtím, že se pustí do pracného převodu u nového řádku, zkontroluje, zda již takový řetězec nepřeváděl. To je pro náš případ, kdy máme hodně dat v omezeném období ideální. Abychom si vyzkoušeli, jak dlouho by převod trval bez cache, nastavíme parametr `cache` na `False`.

In [23]:
invoices["invoice_date_converted"] = pandas.to_datetime(invoices["invoice_date"], format="%d. %m. %Y", cache=False)
invoices.head()

,invoice_date,customer,amount,invoice_date_converted,due_date,status
0,24. 07. 2021,Robbins-Bryant,42424.49,2021-07-24,2021-09-22,before due date
1,03. 06. 2021,"Little, Espinoza and Manning",6269.53,2021-06-03,2021-08-02,overdue
2,24. 06. 2021,Hartman-Garcia,19182.55,2021-06-24,2021-08-23,overdue
3,09. 08. 2021,"Mason, Armstrong and Whitaker",66295.23,2021-08-09,2021-10-08,before due date
4,29. 06. 2021,Dickerson-Blankenship,88592.58,2021-06-29,2021-08-28,overdue


Vidíme, že převod trvá řádově déle, využití vyrovnávací paměti se tedy vyplatí. Pokud bychom např. převáděli 10 milionů záznamů, budou časy převodů (na bězném PC) cca 2 a 20 sekund.